In [1]:
import pandas as pd
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [18]:
def explore_data(df):
    print('shape: {0}'.format(df.shape))
    print('head(3) and tail(3):')
    print(pd.concat([df.head(3), df.tail(3)]))

In [12]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


# Game

Defensive posistions:

| number | position |
| --- | --- |
| 1 | pitcher |
| 2 | catcher |
| 3 | first baseman |
| 4 | second baseman |
| 5 | third baseman |
| 6 | shortstop |
| 7 | left fielder |
| 8 | center fielder |
| 9 | right fielder |

Leagues:

| code | league |
| --- | --- |
| NA | National Association |
| NL | National League |
| AA | American Association |
| UA | Union Association |
| PL | Players League |
| AL | American League |
| FL | Federal League |

In [8]:
game = pd.read_csv('game_log.csv')
explore_data(game)

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


shape: (171907, 161)
head(3) and tail(3):
            date  number_of_game day_of_week v_name v_league  v_game_number  \
0       18710504               0         Thu    CL1      NaN              1   
1       18710505               0         Fri    BS1      NaN              1   
2       18710506               0         Sat    CL1      NaN              2   
171904  20161002               0         Sun    LAN       NL            162   
171905  20161002               0         Sun    PIT       NL            162   
171906  20161002               0         Sun    MIA       NL            161   

       h_name h_league  h_game_number  v_score  h_score  length_outs  \
0         FW1      NaN              1        0        2         54.0   
1         WS3      NaN              1       20       18         54.0   
2         RC1      NaN              1       12        4         54.0   
171904    SFN       NL            162        1        7         51.0   
171905    SLN       NL            162       

# Park

game.park_id = park.park_id

In [9]:
park = pd.read_csv('park_codes.csv')
explore_data(park)

shape: (252, 9)
head(3) and tail(3):
    park_id                                 name  \
0     ALB01                       Riverside Park   
1     ALT01                        Columbia Park   
2     ANA01             Angel Stadium of Anaheim   
249   WOR01   Agricultural County Fair Grounds I   
250   WOR02  Agricultural County Fair Grounds II   
251   WOR03       Worcester Driving Park Grounds   

                               aka       city state       start         end  \
0                              NaN     Albany    NY  09/11/1880  05/30/1882   
1                              NaN    Altoona    PA  04/30/1884  05/31/1884   
2    Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966         NaN   
249                            NaN  Worcester    MA  05/01/1880  09/29/1882   
250                            NaN  Worcester    MA  08/17/1887  08/17/1887   
251                            NaN  Worcester    MA  10/30/1874  10/30/1874   

    league                                  

# Person

game.id (see list below) = person.id
* *_umpire_id
* *_manager_id
* *_pitcher_id
* *_batter_id
* *_player_#_id

In [10]:
person = pd.read_csv('person_codes.csv')
explore_data(person)

shape: (20494, 7)
head(3) and tail(3):
             id      last   first player_debut mgr_debut coach_debut ump_debut
0      aardd001   Aardsma   David   04/06/2004       NaN         NaN       NaN
1      aaroh101     Aaron    Hank   04/13/1954       NaN         NaN       NaN
2      aarot101     Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
20491  zwild101  Zwilling   Dutch   08/14/1910       NaN  04/15/1941       NaN
20492  zycht001      Zych    Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson     NaN          NaN       NaN         NaN       NaN


# Team

game.*_name = team.team_id

In [11]:
team = pd.read_csv('team_codes.csv')
explore_data(team)

shape: (150, 8)
head(3) and tail(3):
    team_id league  start   end        city         nickname franch_id  seq
0       ALT     UA   1884  1884     Altoona  Mountain Cities       ALT    1
1       ARI     NL   1998     0     Arizona     Diamondbacks       ARI    1
2       BFN     NL   1879  1885     Buffalo           Bisons       BFN    1
147     WSN     NL   1892  1899  Washington         Senators       WS9    2
148     WSU     UA   1884  1884  Washington        Nationals       WSU    1
149     MIA     NL   2012     0       Miami          Marlins       FLO    2


In [3]:
db = 'mlb.db'
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
def show_tables():
    q = "select name, type from sqlite_master where type in ('table','view')"
    return run_query(q)
def df_to_table(df, table, exists='fail'):
    with sqlite3.connect(db) as conn:
        df.to_sql(table, conn, index=False, if_exists=exists)

In [29]:
# df_to_table(game, 'game_log')
# df_to_table(park, 'park_codes')
# df_to_table(person, 'person_codes')
# df_to_table(team, 'team_codes')

In [26]:
c_add_game_id = """
    ALTER TABLE game_log
    ADD COLUMN game_id TEXT
    ;
    """
# run_command(c_add_game_id)
c_pop_game_id = """
    UPDATE game_log
    SET game_id = h_name || date || number_of_game
    ;
    """
# run_command(c_pop_game_id)

In [71]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,league,table
7,appearance_type,table
8,team,table
9,game,table


## note: skipping schema design

In [34]:
c_create_person ="""
    CREATE TABLE IF NOT EXISTS person (
        person_id TEXT PRIMARY KEY,
        first_name TEXT,
        last_name TEXT
    );
    """
c_pop_person = """
    INSERT INTO person
    SELECT
        id person_id,
        first first_name,
        last last_name
    FROM
        person_codes
    ;
    """
q_check_person = "select * from person limit 3"
# run_command(c_create_person)
# run_command(c_pop_person)
run_query(q_check_person)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron


In [36]:
c_create_park ="""
    CREATE TABLE IF NOT EXISTS park (
        park_id TEXT PRIMARY KEY,
        name TEXT,
        nickname TEXT,
        city TEXT,
        state TEXT,
        notes TEXT
    );
    """
c_pop_park = """
    INSERT INTO park
    SELECT
        park_id,
        name,
        aka nickname,
        city,
        state,
        notes
    FROM
        park_codes
    ;
    """
q_check_park = "select * from park limit 3"
# run_command(c_create_park)
# run_command(c_pop_park)
run_query(q_check_park)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None


In [39]:
c_create_league ="""
    CREATE TABLE IF NOT EXISTS league (
        league_id TEXT PRIMARY KEY,
        name TEXT
    );
    """
c_pop_league = """
    INSERT INTO league VALUES
    ('NA', 'National Association'),
    ('NL', 'National League'),
    ('AA', 'American Association'),
    ('UA', 'Union Association'),
    ('PL', 'Players League'),
    ('AL', 'American League'),
    ('FL', 'Federal League')
    ;
    """
q_check_league = "select * from league limit 3"
# run_command(c_create_league)
# run_command(c_pop_league)
run_query(q_check_league)

,league_id,name
0,NA,National Association
1,NL,National League
2,AA,American Association


In [79]:
appearance = pd.read_csv('appearance_type.csv')
c_create_appearance_type ="""
    CREATE TABLE IF NOT EXISTS appearance_type (
        appearance_type_id TEXT PRIMARY KEY,
        name TEXT,
        category TEXT
    );
    """
q_check_appearance_type = "select * from appearance_type limit 3"
# run_command(c_create_appearance_type)
# df_to_table(appearance, 'appearance_type', exists='append')
run_query(q_check_appearance_type)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense


In [57]:
c_create_team ="""
    CREATE TABLE IF NOT EXISTS team (
        team_id TEXT PRIMARY KEY,
        league_id TEXT,
        city TEXT,
        nickname TEXT,
        franch_id TEXT,
        FOREIGN KEY (league_id) REFERENCES league(league_id)
    );
    """
c_pop_team = """
    INSERT INTO team
    SELECT DISTINCT
        team_id,
        league league_id,
        city,
        nickname,
        franch_id
    FROM
        team_codes
    WHERE
        NOT (team_id = 'MIL' AND league = 'AL')
    ;
    """
q_check_team = "select * from team limit 3"
# run_command(c_create_team)
# run_command(c_pop_team)
run_query(q_check_team)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN


In [68]:
c_create_game ="""
    CREATE TABLE IF NOT EXISTS game (
        game_id TEXT PRIMARY KEY,
        date TEXT,
        number_of_game TEXT,
        park_id TEXT,
        length_outs INTEGER,
        day BOOLEAN,
        completion TEXT,
        forefeit TEXT,
        protest TEXT,
        attendance INTEGER,
        length_minutes INTEGER,
        additional_info TEXT,
        acquisition_info TEXT,
        FOREIGN KEY (park_id) REFERENCES park(park_id)
    );
    """
c_pop_game = """
    INSERT INTO game
    SELECT
        game_id,
        date,
        number_of_game,
        park_id,
        length_outs,
        CASE
            WHEN day_night = 'D' THEN 1
            ELSE 0
        END as day,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM
        game_log
    ;
    """
q_check_game = "select * from game limit 3"
# run_command(c_create_game)
# run_command(c_pop_game)
run_query(q_check_game)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y


In [77]:
c_create_team_appearance ="""
    CREATE TABLE IF NOT EXISTS team_appearance (
        team_id TEXT,
        game_id TEXT,
        home BOOLEAN,
        league_id TEXT,
        score INTEGER,
        line_score TEXT,
        at_bats INTEGER,
        hits INTEGER,
        doubles INTEGER,
        triples INTEGER,
        homeruns INTEGER,
        rbi INTEGER,
        sacrifice_hits INTEGER,
        sacrifice_flies INTEGER,
        hit_by_pitch INTEGER,
        walks INTEGER,
        intentional_walks INTEGER,
        strikeouts INTEGER,
        stolen_bases INTEGER,
        caught_stealing INTEGER,
        grounded_into_double INTEGER,
        first_catcher_interference INTEGER,
        left_on_base INTEGER,
        pitchers_used INTEGER,
        individual_earned_runs INTEGER,
        team_earned_runs INTEGER,
        wild_pitches INTEGER,
        balks INTEGER,
        putouts INTEGER,
        assists INTEGER,
        errors INTEGER,
        passed_balls INTEGER,
        double_plays INTEGER,
        triple_plays INTEGER,
        PRIMARY KEY (team_id, game_id),
        FOREIGN KEY (team_id) REFERENCES team(team_id),
        FOREIGN KEY (game_id) REFERENCES game(game_id),
        FOREIGN KEY (league_id) REFERENCES league(league_id)
    );
    """
c_pop_team_appearance = """
    INSERT INTO team_appearance
        SELECT
            h_name team_id,
            game_id,
            1 as home,
            h_league league_id,
            h_score score,
            h_line_score line_score,
            h_at_bats at_bats,
            h_hits hits,
            h_doubles doubles,
            h_triples triples,
            h_homeruns homeruns,
            h_rbi rbi,
            h_sacrifice_hits sacrifice_hits,
            h_sacrifice_flies sacrifice_flies,
            h_hit_by_pitch,
            h_walks walks,
            h_intentional_walks intentional_walks,
            h_strikeouts strikeouts,
            h_stolen_bases stolen_bases,
            h_caught_stealing caught_stealing,
            h_grounded_into_double grounded_into_double,
            h_first_catcher_interference first_catcher_interference,
            h_left_on_base left_on_base,
            h_pitchers_used pitchers_used,
            h_individual_earned_runs individual_earned_runs,
            h_team_earned_runs team_earned_runs,
            h_wild_pitches wild_pitches,
            h_balks balks,
            h_putouts putouts,
            h_assists assists,
            h_errors errors,
            h_passed_balls passed_balls,
            h_double_plays double_plays,
            h_triple_plays triple_plays
        FROM
            game_log
            
    UNION
    
        SELECT
            v_name team_id,
            game_id,
            0 as home,
            v_league league_id,
            v_score score,
            v_line_score line_score,
            v_at_bats at_bats,
            v_hits hits,
            v_doubles doubles,
            v_triples triples,
            v_homeruns homeruns,
            v_rbi rbi,
            v_sacrifice_hits sacrifice_hits,
            v_sacrifice_flies sacrifice_flies,
            v_hit_by_pitch,
            v_walks walks,
            v_intentional_walks intentional_walks,
            v_strikeouts strikeouts,
            v_stolen_bases stolen_bases,
            v_caught_stealing caught_stealing,
            v_grounded_into_double grounded_into_double,
            v_first_catcher_interference first_catcher_interference,
            v_left_on_base left_on_base,
            v_pitchers_used pitchers_used,
            v_individual_earned_runs individual_earned_runs,
            v_team_earned_runs team_earned_runs,
            v_wild_pitches wild_pitches,
            v_balks balks,
            v_putouts putouts,
            v_assists assists,
            v_errors errors,
            v_passed_balls passed_balls,
            v_double_plays double_plays,
            v_triple_plays triple_plays
        FROM
            game_log
    ;
    """
q_check_team_appearance = "select * from team_appearance limit 3"
# run_command(c_create_team_appearance)
# run_command(c_pop_team_appearance)
run_query(q_check_team_appearance)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [86]:
c_create_person_appearance ="""
    CREATE TABLE IF NOT EXISTS person_appearance (
        appearance_id INTEGER PRIMARY KEY,
        person_id TEXT,
        team_id TEXT,
        game_id TEXT,
        appearance_type_id TEXT,
        FOREIGN KEY (person_id) REFERENCES person(person_id),
        FOREIGN KEY (team_id) REFERENCES team(team_id),
        FOREIGN KEY (game_id) REFERENCES game(game_id),
        FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
    );
    """
c_pop_person_appearance_non_players_template = """
    INSERT INTO person_appearance (
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
        SELECT
            game_id,
            {team},
            [{field}],
            '{appearance_type}'
        FROM
            game_log
        WHERE
            [{field}] IS NOT NULL
    ;
    """
c_pop_person_appearance_non_players_queries = [
    ['hp_umpire_id', 'UHP', 'NULL'],
    ['1b_umpire_id', 'U1B', 'NULL'],
    ['2b_umpire_id', 'U2B', 'NULL'],
    ['3b_umpire_id', 'U3B', 'NULL'],
    ['lf_umpire_id', 'ULF', 'NULL'],
    ['rf_umpire_id', 'URF', 'NULL'],
    ['v_manager_id', 'MM', 'v_name'],
    ['h_manager_id', 'MM', 'h_name'],
    ['winning_pitcher_id', 'AWP', 'NULL'],
    ['losing_pitcher_id', 'ALP', 'NULL'],
    ['saving_pitcher_id', 'ASP', 'NULL'],
    ['winning_rbi_batter_id', 'AWB', 'NULL'],
    ['v_starting_pitcher_id', 'PSP', 'v_name'],
    ['h_starting_pitcher_id', 'PSP', 'h_name']
]

# run_command(c_create_person_appearance)

for query in c_pop_person_appearance_non_players_queries:
    query_vars = {
        'field': query[0],
        'appearance_type': query[1],
        'team': query[2]
    }
    # run_command(c_pop_person_appearance_non_players_template.format(**query_vars))

q_check_person_appearance_non_players = """
    select * from person_appearance 
    where appearance_type_id in ('UHP','U1B','U2B','U3B','ULF','URF','MM','AWP','ALP','ASP','AWB','PSP')
    and game_id = 'SFN201610020'
    """
run_query(q_check_person_appearance_non_players)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,171886,knigb901,None,SFN201610020,UHP
1,318926,westj901,None,SFN201610020,U1B
2,407466,fleta901,None,SFN201610020,U2B
3,524189,danlk901,None,SFN201610020,U3B
4,696308,robed001,LAN,SFN201610020,MM
5,868215,bochb002,SFN,SFN201610020,MM
6,1008444,moorm003,None,SFN201610020,AWP
7,1148673,maedk001,None,SFN201610020,ALP
8,1302390,poseb001,None,SFN201610020,AWB
9,1474253,maedk001,LAN,SFN201610020,PSP


In [89]:
c_pop_person_appearance_players_template = """
    INSERT INTO person_appearance (
        game_id,
        team_id,
        person_id,
        appearance_type_id
    ) 
        SELECT
            game_id,
            {hv}_name,
            {hv}_player_{num}_id,
            "O{num}"
        FROM
            game_log
        WHERE
            {hv}_player_{num}_id IS NOT NULL

    UNION

        SELECT
            game_id,
            {hv}_name,
            {hv}_player_{num}_id,
            "D" || CAST({hv}_player_{num}_def_pos AS INT)
        FROM
            game_log
        WHERE
            {hv}_player_{num}_id IS NOT NULL
        ;
    """

for hv in ['h','v']:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        #run_command(c_pop_person_appearance_players_template.format(**query_vars))

q_check_person_appearance_players = """
    select * from person_appearance 
    where appearance_type_id not in ('UHP','U1B','U2B','U3B','ULF','URF','MM','AWP','ALP','ASP','AWB','PSP')
    and game_id = 'SFN201610020'
    order by team_id asc, appearance_type_id asc
    """
run_query(q_check_person_appearance_players)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,6672779,maedk001,LAN,SFN201610020,D1
1,5827757,grany001,LAN,SFN201610020,D2
2,5546081,gonza003,LAN,SFN201610020,D3
3,6391109,utlec001,LAN,SFN201610020,D4
4,4701053,turnj001,LAN,SFN201610020,D5
5,4982729,seagc001,LAN,SFN201610020,D6
6,4419377,kendh001,LAN,SFN201610020,D7
7,6109433,pedej001,LAN,SFN201610020,D8
8,5264405,puigy001,LAN,SFN201610020,D9
9,4419378,kendh001,LAN,SFN201610020,O1


In [4]:
tables = ['game_log','park_codes','team_codes','person_codes']
for table in tables:
    c_drop_table_template = """
        DROP TABLE {}
        """
    run_command(c_drop_table_template.format(table))
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table
